# Chapter 12: Quantum Reality

--

**Prerequisites:**
- See `Chapter02_QuantumSoftware.ipynb` for installation instructions


In [1]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from Chapter08_QuantumGates_functions import simulate_statevector, simulate_measurements #type: ignore
from qiskit_aer.noise import NoiseModel, depolarizing_error
from Chapter11_QLA_functions import (LCU_Ax)
from Chapter12_QuantumReality_functions import (postprocess_measurements, LCU_fTAx, execute_fTAx_from_result)


## Ax using PREP-SELECT-UNPREP

In [4]:
A = np.array([
    [1, 0, 0, -0.5],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [-0.5, 0, 0, 1]
])

x = np.array([0.6, 0.8, 0, 0])
expected_output = A @ x

qc, metadata = LCU_Ax(A, x, mode='measurement')

simulator = AerSimulator()
circuit_transpiled = transpile(qc, simulator)
job = simulator.run(circuit_transpiled, shots=10000)
result = job.result()
counts = result.get_counts(qc)

res_vector, success_prob = postprocess_measurements(counts, metadata)
print("Quantum output:", np.round(np.real_if_close(res_vector), 4))
expected_output = A @ x
print("Expected output:", np.round(np.real_if_close(expected_output), 4))
print("Success probability:", np.round(success_prob, 4))



Quantum output: [0.5981 0.8032 0.     0.3154]
Expected output: [ 0.6  0.8  0.  -0.3]
Success probability: 0.4899


## Compute an observale f^T * A*x

In [5]:
f = np.array([1, 0, 0.5, 0.2])
f = f / np.linalg.norm(f)  # Ensure f is normalized
quantum_result, success_prob = LCU_fTAx(f, A, x, shots=10000)
expected = f.T @ A @ x
print(f"Expected magnitude: {expected:.4f}")
print(f"Quantum result: {quantum_result:.4f}")
print(f"Success probability: {success_prob:.4f}")


Expected magnitude: 0.4754
Quantum result: 0.4727
Success probability: 0.4818


## Noise Model

In [ ]:
simulator_ideal = AerSimulator()
qc_trans = transpile(qc.decompose(reps=5), simulator_ideal)

noise_model = NoiseModel()
error = depolarizing_error(0.01, 1)  # 1% single-qubit error
noise_model.add_all_qubit_quantum_error(error, 
    ['u1', 'u2', 'u3', 'x', 'h', 'reset'])
simulator_noisy = AerSimulator(noise_model=noise_model)
result_noisy = simulator_noisy.run(qc_trans, shots=50000).result()
ftax_noisy, prob_noisy = execute_fTAx_from_result(
    result_noisy, metadata)

# Compare results
print(f"Classical:        {np.abs(expected):.4f}")
print(f"Quantum (noisy):  {ftax_noisy:.4f}")
print(f"Noisy error:      {abs(ftax_noisy - np.abs(expected)):.4f}")

Classical:        0.4754
Quantum (noisy):  0.6015
Noisy error:      0.1261
